In [1]:
module GA

# -------

importall Base

export  Entity,
        GAmodel,

        runga,
        freeze,
        defrost,
        generation_num,
        population

# -------

abstract Entity

isless(lhs::Entity, rhs::Entity) = lhs.fitness < rhs.fitness

fitness!(ent::Entity, fitness_score) = ent.fitness = fitness_score

# -------

type EntityData
    entity
    generation::Int

    EntityData(entity, generation::Int) = new(entity, generation)
    EntityData(entity, model) = new(entity, model.gen_num)
end

# -------

type GAmodel
    initial_pop_size::Int
    gen_num::Int
    max_gen::Int

    population::Array
    pop_data::Array{EntityData}
    freezer::Array{EntityData}

    rng::AbstractRNG

    ga

    GAmodel() = new(0, 1, 10000, Any[], EntityData[], EntityData[], MersenneTwister(time_ns()), nothing)
end

global _g_model

# -------

function freeze(model::GAmodel, entity::EntityData)
    push!(model.freezer, entity)
    println("Freezing: ", entity)
end

function freeze(model::GAmodel, entity)
    entitydata = EntityData(entity, model.gen_num)
    freeze(model, entitydata)
end

freeze(entity) = freeze(_g_model, entity)


function defrost(model::GAmodel, generation::Int)
    filter(model.freezer) do entitydata
        entitydata.generation == generation
    end
end

defrost(generation::Int) = defrost(_g_model, generation)


generation_num(model::GAmodel = _g_model) = model.gen_num


population(model::GAmodel = _g_model) = model.population


function runga(mdl::Module; initial_pop_size = 128, max_gen = 10000)
    model = GAmodel()
    model.ga = mdl
    model.initial_pop_size = initial_pop_size
    model.max_gen = max_gen

    runga(model)
end

function runga(model::GAmodel)
    reset_model(model)
    create_initial_population(model)

    while true
        evaluate_population(model)
        if model.gen_num > model.max_gen; break; end
        monitor(model)

        grouper = @task model.ga.group_entities(model.population)
        groupings = Any[]
        while !istaskdone(grouper)
            group = consume(grouper)
            group != nothing && push!(groupings, group)
        end

        if length(groupings) < 1
            break
        end

        crossover_population(model, groupings)
        mutate_population(model)
    end

    model
end

# -------
function reset_model(model::GAmodel)
    global _g_model = model

    model.gen_num = 1
    empty!(model.population)
    empty!(model.pop_data)
    empty!(model.freezer)
end

function create_initial_population(model::GAmodel)
    for i = 1:model.initial_pop_size
        entity = model.ga.create_entity(i)

        push!(model.population, entity)
        push!(model.pop_data, EntityData(entity, model.gen_num))
    end
end

function evaluate_population(model::GAmodel)
    scores = pmap(model.ga.fitness, model.population)
    for i in 1:length(scores)
        fitness!(model.population[i], scores[i])
    end

    sort!(model.population; rev = true)
end

function monitor(model::GAmodel)
  model.ga.monitor(model.population)
end

function crossover_population(model::GAmodel, groupings)
    old_pop = model.population

    model.population = Any[]
    sizehint!(model.population, length(old_pop))

    model.pop_data = EntityData[]
    sizehint!(model.pop_data, length(old_pop))

    model.gen_num += 1

    for group in groupings
        parents = Any[ old_pop[i] for i in group ]
        entity = model.ga.crossover(parents)

        push!(model.population, entity)
        push!(model.pop_data, EntityData(entity, model.gen_num))
    end
end

function mutate_population(model::GAmodel)
    for entity in model.population
        model.ga.mutate(entity)
    end
end

end

GA

In [15]:
using GA

type Net <: Entity
  W::Array{Float64,2}
  b::Array{Float64,1}
  fitness::Float64
  dates::Array{UTF8String,1}

  Net() = begin
    dates=readdir(train_jld_path,Val{:abs})[dates_idx];date=dates[1];
    data,~,~= dataProvider(date);state = load(date,"state");nfeature=size(data,1);
    W=0.001*[zeros(1,nfeature);ones(1,nfeature);zeros(1,nfeature)];b=zeros(3);
    return new(W,b,-10.0,dates)
  end
end

function create_entity(num)
    Net()
end

function group_entities(pop)
    if pop[1].fitness > max_profit
        return
    end

    # freeze(pop[1])

    for i in 1:length(pop)
        produce([1, i])
    end
end

function crossover(group)
    child = Net()

    num_parents = length(group)
    for I in CartesianRange(size(group[1].W))
        parent = (rand(UInt) % num_parents) + 1
        child.W[I] = group[parent].W[I]
    end
    for I in CartesianRange(size(group[1].b))
        parent = (rand(UInt) % num_parents) + 1
        child.b[I] = group[parent].b[I]
    end

    child
end

function mutate(net;σ=0.01)
    # let's go crazy and mutate 20% of the time
    rand(Float64) < 0.8 && return

    BLAS.axpy!(σ,randn(size(net.W)),net.W)
    BLAS.axpy!(σ,randn(size(net.b)),net.b)
end

function monitor(population;σ=0.01)
  println(population[1].fitness," ",population[end].fitness)
end


  likely near In[15]:3


monitor (generic function with 1 method)

In [8]:
addprocs(2)
runga(testga; initial_pop_size = 8)

LoadError: LoadError: UndefVarError: nprocs_to_add not defined
while loading In[8], in expression starting on line 1

In [ ]:
@everywhere begin
  using GeneticAlgorithms
  include("linear.jl")
end
model = runga(Linear; initial_pop_size = 16)

dates=Linear.readdir(Linear.train_jld_path,Val{:abs})[1:10];
Linear.fitness(model.population[1],dates)
